In [3]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import math
import readline
import re 
import pandas as pd 
from bs4 import BeautifulSoup
import random
from google.colab import drive




In [5]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 180 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=1b8a4bd188f2d00be9eeb2eb9af6d04c155a64629315751beac0b6cb5f6146fb
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=6e4cfd8613429b5423859fa090ea86f2d810d5fbed7c278779aeb061f7f54cf5
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=bc2c321b22249f1b696557c9dcf40b2795d006a1b3d94b8384ad6796771e3843
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow p

In [6]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert




Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [7]:
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
cols= ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv("/content/drive/MyDrive/bert_prac/training.csv",
                   header= None, 
                   names = cols,
                   engine ="python", 
                   encoding= "latin1")

In [9]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [10]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [11]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+",' ',tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ',tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ',tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet
    


In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]


In [13]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] =1


In [14]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case =bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [15]:
tokenizer.tokenize("my dog loves travelling and  strawberries.")


['my', 'dog', 'loves', 'travelling', 'and', 'straw', '##berries', '.']

In [16]:
def encode_sentence(line):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(line))
    

In [17]:
data_input = [encode_sentence(sentence)for sentence in data_clean]


In [18]:
data_input

[[22091,
  2860,
  2860,
  2008,
  1005,
  1055,
  1037,
  26352,
  5017,
  1012,
  2017,
  2323,
  2050,
  2288,
  2585,
  12385,
  1997,
  2353,
  2154,
  2000,
  2079,
  2009,
  1012,
  1040],
 [2003,
  6314,
  2008,
  2002,
  2064,
  1005,
  1056,
  10651,
  2010,
  9130,
  2011,
  3793,
  2075,
  2009,
  1012,
  1012,
  1012,
  1998,
  2453,
  5390,
  2004,
  1037,
  2765,
  2082,
  2651,
  2036,
  1012,
  27984,
  999],
 [1045,
  11529,
  2094,
  2116,
  2335,
  2005,
  1996,
  3608,
  1012,
  3266,
  2000,
  3828,
  1996,
  2717,
  2175,
  2041,
  1997,
  19202],
 [2026, 2878, 2303, 5683, 2009, 11714, 1998, 2066, 2049, 2006, 2543],
 [2053,
  2009,
  1005,
  1055,
  2025,
  2022,
  3270,
  6455,
  2012,
  2035,
  1012,
  1045,
  1005,
  1049,
  5506,
  1012,
  2339,
  2572,
  1045,
  2182,
  1029,
  2138,
  1045,
  2064,
  1005,
  1056,
  2156,
  2017,
  2035,
  2058,
  2045,
  1012],
 [2025, 1996, 2878, 3626],
 [2342, 1037, 8549],
 [4931,
  2146,
  2051,
  2053,
  2156,
  999,
 

In [19]:
data_with_len = [[sent,data_labels[i],len(sent)]
                 for i, sent in enumerate(data_input)]
random.shuffle(data_with_len)

data_with_len.sort(key= lambda x: x[2])
sorted_all = [(sent_lab[0],sent_lab[1])
                for sent_lab in data_with_len if sent_lab[2]>7]


In [20]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types= (tf.int32, tf.int32))

In [21]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([ 2145, 21122,  5999,  2039,  1998,  1055, 17048,  3723],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [22]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [23]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 2145, 21122,  5999,  2039,  1998,  1055, 17048,  3723],
        [ 4931,  2008, 12669,  7908,  2003,  2033,  1012,  1012],
        [ 2288,  5003,  3042,  2067, 19658,  3398, 23644,  2232],
        [ 2003,  2045,  2505,  2017,  3685, 20137,  5562,  1029],
        [28983,  2672,  2017,  5293,  2017,  2128,  1999,  2710],
        [25730, 11350,  1998, 20548,  7852,  2003,  2005,  4596],
        [ 8840,  2140,  2053,  2665, 16215, 29232,  2182,   999],
        [17055, 10047,  2007,  1057,  2006,  2008,   999,  1060],
        [ 1045,  2074,  2288,  1996,  8484,  4602,  2132, 21881],
        [ 6160,  2000,  1996,  2047, 17418,  2124,  2004, 10474],
        [ 2821,  1045,  4299,  2026,  6203,  2745,  1045,  4299],
        [ 2049,  2061,  2980,  2182,  1998, 10047,  7501,   999],
        [22482,  2003,  2066,  2026,  1047,  2854, 15857,  4221],
        [ 2293,  2422,  1998, 24618,  2067,  2000,  2017,   999],
        [10047,  2074,  5962

In [24]:
NO_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NO_BATCHES_TEST =NO_BATCHES// 10
all_batched.shuffle(NO_BATCHES)
test_dataset = all_batched.take(NO_BATCHES_TEST)
train_dataset= all_batched.skip(NO_BATCHES_TEST)



In [25]:
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                 enb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training= False,
                 name='dcnn'
                 ):
        super(DCNN,self).__init__(name=name)

        self.embedding= layers.Embedding(vocab_size,enb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=3,
                                    padding="valid",
                                    activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=4,
                                    padding="valid",
                                    activation="relu")
        self.pool = layers.GlobalMaxPooling1D()
        self.dense_1= layers.Dense(units =FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if(nb_classes==2):
            self.last_dense = layers.Dense(units =1 ,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units =nb_classes ,
                                           activation="softmax")
    def call(self, inputs, training):
        x=self.embedding(inputs)
        x_1=self.bigram(x)
        x_1=self.pool(x_1)
        x_2=self.trigram(x)
        
        x_2=self.pool(x_2)
        x_3=self.fourgram(x)
        x_3=self.pool(x_3)

        merged = tf.concat([x_1,x_2,x_3], axis =-1)
        merged = self.dense_1(merged)
        merged = self.dropout(merged,training)
        output = self.last_dense(merged)

        return output




        

        
        

In [26]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FIlters =100
FFN_UNITS = 256
NB_CLASSES =2
DROPOFF_RATE = 0.2
NB_EPOCES = 5


In [27]:
Dcnn =DCNN(vocab_size=VOCAB_SIZE,
           enb_dim=EMB_DIM,
           nb_filters=NB_FIlters,
           FFN_units=FFN_UNITS,
           nb_classes=NB_CLASSES,
           dropout_rate=DROPOFF_RATE)

In [28]:
from tensorflow.python import metrics
if NB_CLASSES== 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer ="adam",
                 metrics=["sparse_categorical_accuracy"])
    


In [29]:
checkpoint_path = "/content/drive/MyDrive/bert_prac/ckpt"
ckpt =tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager =tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep =1 )

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("latest ckpt restored")
    

In [30]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self,epoch, logs= None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [31]:

Dcnn.fit(train_dataset, 
         epochs=NB_EPOCES,
         callbacks=[MyCustomCallback()])


Epoch 1/5
  37196/Unknown - 484s 12ms/step - loss: 0.4296 - accuracy: 0.8021Checkpoint saved at /content/drive/MyDrive/bert_prac/ckpt.
37196/37196 [==============================] - 484s 12ms/step - loss: 0.4296 - accuracy: 0.8021
Epoch 2/5
37196/37196 [==============================] - 456s 12ms/step - loss: 0.3822 - accuracy: 0.8299
Epoch 3/5
37196/37196 [==============================] - 450s 12ms/step - loss: 0.3426 - accuracy: 0.8508
Epoch 4/5
37196/37196 [==============================] - 445s 12ms/step - loss: 0.3026 - accuracy: 0.8704
Epoch 5/5
37196/37196 [==============================] - 446s 12ms/step - loss: 0.2651 - accuracy: 0.8878


In [32]:
results= Dcnn.evaluate(test_dataset)
print(results)

4132/4132 [==============================] - 31s 7ms/step - loss: 0.4261 - accuracy: 0.8260
[0.4260600209236145, 0.8260073661804199]


In [45]:
def get_prediction(sentence):
    tokens= encode_sentence(sentence)
    inputs= tf.expand_dims(tokens,0)
    output = Dcnn(inputs, training= False)
    sentiment = math.floor(output*2)
    if(sentiment ==0 ):
        print("the sentence is negative")
    else:
        print("the sentence is positive")

In [38]:
sentence = "The overall performance of out team was very poor in ICC T20 world cup"
get_prediction(sentence)

negative


In [44]:
sentence = "The shawshank redemption was pretty interesting"
get_prediction(sentence)

positive


In [41]:
pip install git+https://github.com/csebuetnlp/normalizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-fq5odcf9
  Running command git clone -q https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-fq5odcf9
     |████████████████████████████████| 184 kB 32.6 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6887 sha256=027b8fdf654c594b76283b44ec78c6d9ec0e059da1b72e520530758acf8ffac6
  Stored in directory: /tmp/pip-ephem-wheel-cache-kztu3o2r/wheels/af/b1/ee/b9e2a2f2dd861976a357b6a6fa105aeedf2254016676f6cf8f
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=75f5dec059dfb22e854ca8c41439dda19bd1faa429d4550a1b438389100c9675
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl siz

In [43]:
pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 13.1 MB/s 
     |████████████████████████████████| 132 kB 55.5 MB/s 
     |████████████████████████████████| 79 kB 10.3 MB/s 
     |████████████████████████████████| 9.9 MB 64.0 MB/s 
     |████████████████████████████████| 140 kB 70.5 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
